In [2]:
import nltk
nltk.download('webtext')
from nltk import bigrams, trigrams
from nltk.corpus import webtext
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize
import string
import random

[nltk_data] Downloading package webtext to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package webtext is already up-to-date!


In [28]:
txtfile = open('sam_content.txt', 'r', encoding='utf-8')
docs = []
for line in txtfile:
    docs.append(line)

In [29]:
print(docs[0])

Lionel Messi isn't just the world's greatest soccer player, he's also something of an interesting fashion case. From the polka-dot suit he donned on Monday to accept his fourth Ballon d'Or to the floppy Beatles-esque haircut he rocked in his earlier days to his frequent resemblance of a teenage skater, the Argentine goal machine keeps us entertained with more than just his on-field heroics. SEE ALSO: Messi's Year of Dominance: Watch All 91 Goals in One Epic MashupBuckle your Gucci seatbelt and take a ride through some of Messi's greatest fashion triumphs below. 1. The 'When You Score 91 Goals in a Calendar Year, You Can Wear Whatever You Damn Well Please' LookMessi was resplendent in this reverse-dalmatian polka-dot number when accepting the Ballon d'Or on Monday, sparking plenty of chatter and snide comments on Twitter. But when you score goals at the record-setting rate he did in 2012, you've pretty much earned the right to do whatever you want. (Getty Images/AFP/Fabrice Coffrini)2. 

In [30]:
# split into words
import re
clean_docs = []
table = str.maketrans('', '', string.punctuation)
for line in docs:
    tokens = re.split('(\W)', line)
    words = filter(lambda x:x!=' ', tokens)
    words = filter(None, words)
    clean_docs.append(list(words))

In [37]:
#print(clean_docs[0])

In [32]:
model = defaultdict(lambda: defaultdict(lambda: 0))

for doc in clean_docs:
    for w1,w2,w3 in trigrams(doc, pad_right=True, pad_left=True):
        model[(w1,w2)][w3] += 1
            
for w1,w2 in model:
    total_count = float(sum(model[w1,w2].values()))
    for w3 in model[w1,w2]:
        model[w1,w2][w3] /= total_count
        

In [33]:
import random
import time
random.seed(time.time())
txtfile = open('sam_content_phoney.txt', 'w', encoding='utf-8')

Apostrophe = ["'", '.', ',', '?', '!', ]

for i in range(len(clean_docs)):
    text = [None, None]

    sentence_finished = False

    while not sentence_finished:
        r = random.random()
        accumulator = .0

        word_prev = None
        for word in model[tuple(text[-2:])].keys():
            accumulator += model[tuple(text[-2:])][word]

            if accumulator >= r:
                text.append(word)
                break

            word_prev = word

        if text[-2:] == [None, None]:
            sentence_finished = True
            
    final_sentence = ''
    for word in text:
        
        if word in Apostrophe:
            final_sentence = final_sentence[:-1] + word
        elif len(final_sentence) > 0 and word and final_sentence[-1] == "'":
            final_sentence = final_sentence + word
        elif word:
            final_sentence = final_sentence + ' ' + word
            
    # remove trailing space
    final_sentence = final_sentence[:-1]
    txtfile.write(final_sentence + '\n')
txtfile.close()

 Nick Youn? ( Laughs ) But Mess's top pick in last yea, I figured I would never drive thru watts " — that you have to be running an experiment to see who ranks wher. NBAWhen it comes to NFL owners — the money we want to hit War, J, preens after defeating Juan Manual Marquez on Oc. 1. Olympic Star Tom Daley Outs a Hateful Troll on Twitte. Hi everybod. Go get i, howeve, has more than 1. Judgment FailBut that was't change any of the Seattle Post - Standar's Funniest Sports FailsSports Fails. Golf Cart FailThis runaway vehicle wreaked minor on - field havoc following a high school senior from Virginia wh's couga. Bonus points for the first time he accidentally gave us his net worth is estimated at nearly $ 90, 000 - meter hurdles championship in Moscow on Au. 1. Image : Al Bello / Getty Images1. David Luiz of Brazil is ecstatic after a teammat's Funniest Sports Freak - outs. Handball Player Drops His Shorts an, as w're gonna pump up the fieldSpai's burgeoning tech community — Google says i

In [36]:
# simple compare
txtfile = open('neha_content_phoney.txt', 'r', encoding='utf-8')
txtfile.readline()
txtfile.close()

In [71]:
# make doc2vec stuff here
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import gensim.downloader as api
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.corpus import stopwords
import csv
stop_words = set(stopwords.words('english'))

In [72]:
# Read content corpus into a dataset list
def clean(docs):
    import re
    clean_docs = []
    table = str.maketrans('', '', string.punctuation)
    for line in docs:
        tokens = re.split('(\W)', line)
        words = filter(lambda x:x!=' ', tokens)
        words = filter(None, words)
        clean_docs.append(list(words))

    return clean_docs

corpus_real = open('real/stan_content.txt', 'r', encoding='utf-8')
corpus_fake = open('fake/stan_content_phoney.txt', 'r', encoding='utf-8')

docs_real = []
docs_fake = []

for line in corpus_real:
    docs_real.append(line)
    
for line in corpus_fake:
    docs_fake.append(line)
    
docs_real = clean(docs_real)
docs_fake = clean(docs_fake)

In [73]:
dataset = []
i = 0
for doc in docs_real:
    dataset.append(gensim.models.doc2vec.TaggedDocument(doc, [1, i]))
    i += 1
for line in docs_fake:
    dataset.append(gensim.models.doc2vec.TaggedDocument(doc, [0, i]))
    i += 1
corpus_file.close()

In [74]:
# create model and make vocab
d2v_model = gensim.models.doc2vec.Doc2Vec(vector_size=50,
                                          min_count=2,
                                          epochs=40)
d2v_model.build_vocab(dataset)

In [75]:
# train model
%time d2v_model.train(dataset,total_examples=d2v_model.corpus_count,epochs=d2v_model.epochs)

Wall time: 13.2 s


In [76]:
# store doc2vec model
d2v_model.save('real/stan_content.model')

In [77]:
# add infered vectors
# make a dict where the author sim to main author ratio is saved
# this takes a while to run

vector_real = open('real/stan_vector.txt', 'w', encoding='utf-8')
vector_fake = open('fake/stan_phoney_vector.txt', 'w', encoding='utf-8')

for i in range(len(dataset)):
    inferred_vector = d2v_model.infer_vector(dataset[i].words)
    rf = dataset[i].tags[0]
    if rf:
        
        vector_real.write(str(inferred_vector) + '\n')
    else:
        vector_fake.write(str(inferred_vector) + '\n')

vector_real.close()
vector_fake.close()